In [ ]:
import intake
import pandas as pd

df = intake.open_csv('./data/bird_migration/{species}.csv').read()

def fill_day(v):
    next_year = v.assign(day=v.day + v.day.max())
    last_year = v.assign(day=v.day - v.day.max())
    surrounding_years = pd.concat([last_year, v, next_year])
    filled = surrounding_years.assign(
        lat=surrounding_years.lat.interpolate(), 
        lon=surrounding_years.lon.interpolate())
    this_year = filled[filled.day.isin(v.day)]
    return this_year

df = pd.concat([fill_day(v) for k, v in df.groupby('species')])

## Holoviews

All of the plots that we made with `hvplot` in the last notebook are actually `holoviews` objects rendered in bokeh. `holoviews` is centered around the idea of annotating your data and letting it display itself. Here we will quickly go through some of the same workflows as we did in the last notebook, but using pure `holoviews`

In [ ]:
import holoviews as hv
hv.extension('bokeh')

### Data structures

`holoviews` has it's own concept of a dataset which can accept lots of different types of array-like data such as `numpy.array`, `pandas.Dataframe`, `xarray.Dataset`. To create a `holoviews.Dataset` we'll define which of the columns in our dataframe are the *key dimensions* and which are the *value dimensions*.

In [ ]:
bird_ds = hv.Dataset(df, kdims=['lon', 'lat'], vdims=['day', 'species'])

In [ ]:
bird_ds

In [ ]:
bird_ds.data.head()

So we've essentially wrapped some metadata around our data.

### Visualizing data

Since we annotated our data with which dimensions are keys and which are value dimensions, we don't have to specify that later in the call to plot the data. 

In [ ]:
%%opts Points [height=500 width=700]

p = bird_ds.to(hv.Points)
p

In [ ]:
print(p)

We can look at the bird density across the timespan by instead making a hextiles plot.

In [ ]:
%%opts HexTiles [height=500 width=700]

hv.HexTiles(bird_ds)

### Styling

We can define a custom colormap to get the maximum visual distance between each species. Here I am using [Glasbey colors](https://github.com/taketwo/glasbey).

In [ ]:
colors = pd.read_csv('./assets/colormap.csv', header=None, names=['R', 'G', 'B'])
colors.head()

In [ ]:
species_cmap = dict(zip(df.species.cat.categories, 
                    ['#{row.R:02x}{row.G:02x}{row.B:02x}'.format(row=row) 
                     for _, row in colors.iterrows()]))

In [ ]:
%%opts Points [color_index='species' height=500 width=800,legend_position='right'] (cmap=species_cmap)

p

Notice that `p` itself was not altered, just the way that `p` was rendered in that one cell.

### Grouping

Just as we did in `hvplot` we can group these plots and add widgets to enhance the dimensionality of out visualization.

In [ ]:
%%opts Points [color_index='species' height=500 width=700 show_legend=False tools=['hover', 'tap', 'box_select']] 
%%opts Points (cmap=species_cmap size=5)

grouped_birds = p.groupby('day')
grouped_birds

In [ ]:
print(grouped_birds)

### Adding another layer of data

It is often useful to add another layer of information under a dataset. In this case we might suspect that birds are motivated to migrate because of changing temperature. We will explore that hypothesis by importing data from a global climate model. For this we will use `xarray`.

In [ ]:
import xarray as xr
import hvplot.xarray

In [ ]:
ds = xr.open_dataset('http://www.esrl.noaa.gov/psd/thredds/dodsC/Datasets/ncep/air.day.ltm.nc')
ds

The time is in a strange format because it is day of year means for each grid cell. We can convert those to integer day of year like we have for our bird data.

In [ ]:
ds = ds.rename(time='day')
ds['day'] = list(range(1,366))
ds.day

We'll use `hvplot` to quickly take a look at this new data.

In [ ]:
ds['air'].hvplot(x='lon', y='lat', groupby=['level', 'day'], height=500)

Since we are interested in the temperature near the surface of the earth, we only really need the 1000mbar level. We can select that directly from the `xarray.dataset`.

In [ ]:
grouped_air = ds.sel(level=1000).hvplot('lon', 'lat', groupby='day')
grouped_air

In [ ]:
print(grouped_air)

**NOTE:** `hvplot` defaults to dynamic mapping rather than pre-computed mapping. This makes it much quicker to render, but means that all the plots aren't computed ahead of time, so there is a bit of a lag sometimes.

We can explore how this works by dragging the `day` slider and checking the keys that we have on our plot:

In [ ]:
grouped_air.keys()

Since the bird plot and the air plot are both holoviews object on the same axes, we can combine them into one plot. Remember that the air temperature plot was made using `hvplot` and the bird plot was made using `holoviews` - but since the outputs are all holoviews objects this history doesn't matter.

In [ ]:
%%opts Image [height=400]

grouped_air * grouped_birds

Hmmm. That doesn't look great. Turns out that the birds and the temperature use different conventions for longitude. This is a great time to realize that *all* of these data really belong in a geographic context.

[Next Section](./03_geoviews.ipynb#Geoviews)